In [1]:
import re
import os

In [12]:
root_dir = r'G:\nengjing_data'
wget_dir_files={}

for dirpath, dirnames, filenames in os.walk(root_dir):
    if len(filenames) > 0:
        wget_dir_files[dirpath]=f'{dirpath}\\{filenames[0]}'
wget_dir_files

{'G:\\nengjing_data\\huss\\ssp126': 'G:\\nengjing_data\\huss\\ssp126\\wget_script_2025-8-1_21-11-10.sh',
 'G:\\nengjing_data\\huss\\ssp245': 'G:\\nengjing_data\\huss\\ssp245\\wget_script_2025-8-1_21-12-30.sh',
 'G:\\nengjing_data\\huss\\ssp585': 'G:\\nengjing_data\\huss\\ssp585\\wget_script_2025-8-1_21-13-54.sh',
 'G:\\nengjing_data\\mrro\\ssp126': 'G:\\nengjing_data\\mrro\\ssp126\\wget_script_2025-8-1_21-19-37.sh',
 'G:\\nengjing_data\\mrro\\ssp245': 'G:\\nengjing_data\\mrro\\ssp245\\wget_script_2025-8-1_21-20-21.sh',
 'G:\\nengjing_data\\mrro\\ssp585': 'G:\\nengjing_data\\mrro\\ssp585\\wget_script_2025-8-1_21-20-47.sh',
 'G:\\nengjing_data\\rsds\\ssp126': 'G:\\nengjing_data\\rsds\\ssp126\\wget_script_2025-8-1_21-14-59.sh',
 'G:\\nengjing_data\\rsds\\ssp245': 'G:\\nengjing_data\\rsds\\ssp245\\wget_script_2025-8-1_21-15-47.sh',
 'G:\\nengjing_data\\rsds\\ssp585': 'G:\\nengjing_data\\rsds\\ssp585\\wget_script_2025-8-1_21-16-26.sh',
 'G:\\nengjing_data\\rsdscs\\ssp126': 'G:\\nengjing_dat

In [18]:
def process_each_sh(file_path, dir):
    dir_lis=dir.split('\\')
    variant=dir_lis[-2]
    senario=dir_lis[-1]
    with open(file_path, 'r') as f:
        content = f.read()
        patten0 = "http://.*.nc"
        patten1 = "https://.*.nc"

        result0 = re.findall(patten0, content)
        result1 = re.findall(patten1, content)

        with open("{}\\CMIP6LinkGetresult_{}_{}.txt".format(dir, variant, senario), 'w')as wf:
            wf.write('\n'.join(result0))
            wf.write('\n'.join(result1))
#!pip install tqdm
from tqdm import tqdm
for k, v in wget_dir_files.items():
    process_each_sh(v, k)

从转化好的链接下载数据

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
import time
from tqdm import tqdm

In [ ]:
def set_logging(log_file_path='logs/download.log'):
    os.makedirs(os.path.dirname(log_file_path), exist_ok=True)
    # 创建日志记录器
    logger = logging.getLogger('download_logger')
    logger.setLevel(logging.INFO)
    
    # 避免重复添加处理器
    if not logger.handlers:
        # 创建文件处理器
        file_handler = logging.FileHandler(log_file_path, encoding='utf-8')
        file_handler.setLevel(logging.INFO)
        
        # 创建控制台处理器
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        
        # 创建格式器并添加到处理器
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)
        console_handler.setFormatter(formatter)
        
        # 添加处理器到记录器
        logger.addHandler(file_handler)
        logger.addHandler(console_handler)
    
    return logger
    
logger = set_logging()

def load_from_url(url,save_dir,file_name):
    try:
        resp=requests.get(url,stream=True)
        resp.raise_for_status()
        
        os.makedirs(os.path.dirname(save_dir), exist_ok=True)
        
        with open(file_name, 'wb')as f:
            for chunk in tqdm(resp.iter_content(chunk_size=8192),unit='B',unit_divisor=1024*1024, unit_scale=True, desc='下载进度'):
                f.write(chunk)
        logger.info('       ——{} | 成功下载.NC文件：{}'.format(time.time(), file_name))
        return file_name
    except Exception as e:
        logger.error('下载文件 {} 时出错：{} | 下载链接：{}'.format(file_name,e,url))
        return None
    
  
def get_url_from_txt(file_path):
    '''
    Param: file_path | str
    Return: urls | list
    '''
    with open(file_path, 'r') as f:
        urls=[]
        for url in f.readlines():
            urls.append(url.strip('\n'))
    return urls


def main():
    dir_lis=[]
    for k, _ in wget_dir_files.items():
        dir_lis.append(k)
        
    txt_lis = []
    for d in dir_lis:
        for f in os.listdir(d):
            if f.endswith('.txt'):
                txt_lis.append(os.path.join(d, f))
                
    logger.info('======下载.nc文件任务启动======')
    for idx, txt in enumerate(txt_lis): 
        logger.info('   ——处理{}中的链接'.format(txt))
        d = dir_lis[idx]+'\\{}_{}'.format((txt.split('\\')[-1]).split('_')[-2],
                                       (txt.split('\\')[-1]).split('_')[-1])           
        urls = get_url_from_txt(txt)
        with ThreadPoolExecutor(max_workers=len(urls)) as executor:
            futures=[]
            for i in range(len(urls)):
                count=0
                future = executor.submit(load_from_url, urls[i], d, urls[i].split('/')[-1])
                futures.append(future)
            for future in futures:                
                if future.result() is not None:
                    logger.info('       ——{} | 成功下载.NC文件：{}'.format(time.time(),future.result()))
                else:
                    logger.error('      ——{} | 下载失败.NC文件：{}'.format(time.time(), urls[i]))
                    
    logger.info('======下载.nc文件任务结束======')
main()

2025-08-01 23:38:34,164 - INFO - ======下载.nc文件任务启动======
2025-08-01 23:38:34,165 - INFO -    ——处理G:\nengjing_data\huss\ssp126\CMIP6LinkGetresult_huss_ssp126.txt中的链接
下载进度: 0.00B [00:00, ?B/s]





下载进度: 2.00B [00:00, 5.89B/s]
































下载进度: 6.00B [00:01, 4.54B/s]































下载进度: 10.0B [00:01, 5.12B/s]






















































































下载进度: 12.0B [00:03, 2.80B/s]























































































下载进度: 14.0B [00:05, 2.03B/s]





























































































































下载进度: 16.0B [00:07, 1.46B/s]












































下载进度: 18.0B [00:08, 1.55B/s]




















































下载进度: 20.0B [00:09, 1.63B/s]





















































































































In [ ]:
import os
def get_url_from_txt(file_path):
    '''
    Param: file_path | str
    Return: urls | list
    '''
    with open(file_path, 'r') as f:
        urls=[]
        for url in f.readlines():
            urls.append(url.strip('\n'))
    return urls
def wget_files():
    root_dir = r'G:\nengjing_data'
    wget_dir_files={}

    for dirpath, dirnames, filenames in os.walk(root_dir):
        if len(filenames) > 0:
            wget_dir_files[dirpath]=f'{dirpath}\\{filenames[0]}'
    return wget_dir_files

wget_dir_files=wget_files()
dir_lis=[]
for k, _ in wget_dir_files.items():
    dir_lis.append(k)
        
txt_lis = []
for d in dir_lis:
    for f in os.listdir(d):
        if f.endswith('.txt'):
            txt_lis.append(os.path.join(d, f))
            

for idx, txt in enumerate(txt_lis): 
    print('====',idx,'\t',txt,'====')
    d = dir_lis[idx]+'\\{}_{}'.format((txt.split('\\')[-1]).split('_')[-2],
                                    ((txt.split('\\')[-1]).split('_')[-1]).split('.')[0])           
    urls = get_url_from_txt(txt)
    
    pending_urls = []
    for url in urls:
        file_name = url.split('/')[-1]
        pending_urls.append(url)
        
    for i in range(9):
        print('url:\t',pending_urls[i],'\nsave_dir:\t', d,'\nfile_name:\t', pending_urls[i].split('/')[-1])

==== 0 	 G:\nengjing_data\huss\ssp126\CMIP6LinkGetresult_huss_ssp126.txt ====
url:	 http://crd-esgf-drc.ec.gc.ca/thredds/fileServer/esgA_dataroot/AR6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp126/r1i1p2f1/3hr/huss/gn/v20190429/huss_3hr_CanESM5_ssp126_r1i1p2f1_gn_207101010300-208101010000.nc 
save_dir:	 G:\nengjing_data\huss\ssp126\huss_ssp126 
file_name:	 huss_3hr_CanESM5_ssp126_r1i1p2f1_gn_207101010300-208101010000.nc
url:	 http://crd-esgf-drc.ec.gc.ca/thredds/fileServer/esgA_dataroot/AR6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp126/r1i1p2f1/3hr/huss/gn/v20190429/huss_3hr_CanESM5_ssp126_r1i1p2f1_gn_201501010300-202101010000.nc 
save_dir:	 G:\nengjing_data\huss\ssp126\huss_ssp126 
file_name:	 huss_3hr_CanESM5_ssp126_r1i1p2f1_gn_201501010300-202101010000.nc
url:	 http://crd-esgf-drc.ec.gc.ca/thredds/fileServer/esgA_dataroot/AR6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp126/r1i1p2f1/3hr/huss/gn/v20190429/huss_3hr_CanESM5_ssp126_r1i1p2f1_gn_202101010300-203101010000.nc 
save_dir:	 G:\nengjing_data\huss\ssp12

In [4]:
for i in range(len(txt_lis)):
    print('dir:\t',dir_lis[i],'\t|\ttxt:\t',txt_lis[i],'\n')

dir:	 G:\nengjing_data\huss\ssp126 	|	txt:	 G:\nengjing_data\huss\ssp126\CMIP6LinkGetresult_huss_ssp126.txt 

dir:	 G:\nengjing_data\huss\ssp126\huss_ssp126 	|	txt:	 G:\nengjing_data\huss\ssp245\CMIP6LinkGetresult_huss_ssp245.txt 

dir:	 G:\nengjing_data\huss\ssp126\huss_ssp126\huss_ssp245 	|	txt:	 G:\nengjing_data\huss\ssp585\CMIP6LinkGetresult_huss_ssp585.txt 

dir:	 G:\nengjing_data\huss\ssp245 	|	txt:	 G:\nengjing_data\mrro\ssp126\CMIP6LinkGetresult_mrro_ssp126.txt 

dir:	 G:\nengjing_data\huss\ssp245\huss_ssp585 	|	txt:	 G:\nengjing_data\mrro\ssp245\CMIP6LinkGetresult_mrro_ssp245.txt 

dir:	 G:\nengjing_data\huss\ssp585 	|	txt:	 G:\nengjing_data\mrro\ssp585\CMIP6LinkGetresult_mrro_ssp585.txt 

dir:	 G:\nengjing_data\huss\ssp585\mrro_ssp126 	|	txt:	 G:\nengjing_data\rsds\ssp126\CMIP6LinkGetresult_rsds_ssp126.txt 

dir:	 G:\nengjing_data\mrro\ssp126 	|	txt:	 G:\nengjing_data\rsds\ssp245\CMIP6LinkGetresult_rsds_ssp245.txt 

dir:	 G:\nengjing_data\mrro\ssp126\mrro_ssp245 	|	txt:	 G:\n

In [ ]:

import os
def wget_files():
    root_dir = r'G:\nengjing_data'
    wget_dir_files={}

    for dirpath, dirnames, filenames in os.walk(root_dir):
        if len(filenames) > 0:
            for fl in filenames:
                if fl.endswith('.txt'):
                    wget_dir_files[f'{dirpath}\\{fl.split('_')[1]}_{((fl.split('_')[2]).split('.'))[0]}']=(os.path.join(dirpath,fl))
    return wget_dir_files

dic=wget_files()
for k, v in dic.items():
    print('dir\t',k,'\nfile\t',v)

dir	 G:\nengjing_data\huss\ssp126\huss_ssp126 
file	 G:\nengjing_data\huss\ssp126\CMIP6LinkGetresult_huss_ssp126.txt
dir	 G:\nengjing_data\huss\ssp245\huss_ssp245 
file	 G:\nengjing_data\huss\ssp245\CMIP6LinkGetresult_huss_ssp245.txt
dir	 G:\nengjing_data\huss\ssp585\huss_ssp585 
file	 G:\nengjing_data\huss\ssp585\CMIP6LinkGetresult_huss_ssp585.txt
dir	 G:\nengjing_data\mrro\ssp126\mrro_ssp126 
file	 G:\nengjing_data\mrro\ssp126\CMIP6LinkGetresult_mrro_ssp126.txt
dir	 G:\nengjing_data\mrro\ssp245\mrro_ssp245 
file	 G:\nengjing_data\mrro\ssp245\CMIP6LinkGetresult_mrro_ssp245.txt
dir	 G:\nengjing_data\mrro\ssp585\mrro_ssp585 
file	 G:\nengjing_data\mrro\ssp585\CMIP6LinkGetresult_mrro_ssp585.txt
dir	 G:\nengjing_data\rsds\ssp126\rsds_ssp126 
file	 G:\nengjing_data\rsds\ssp126\CMIP6LinkGetresult_rsds_ssp126.txt
dir	 G:\nengjing_data\rsds\ssp245\rsds_ssp245 
file	 G:\nengjing_data\rsds\ssp245\CMIP6LinkGetresult_rsds_ssp245.txt
dir	 G:\nengjing_data\rsds\ssp585\rsds_ssp585 
file	 G:\nengjing

MIROC- 将链接转化为txt


In [36]:
var_lis_group = ['huss','tas','uas','vas']

In [37]:
example_url = 'http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2H/historical/r1i1p4f2/3hr/huss/gn/v20230904/huss_3hr_MIROC-ES2H_historical_r1i1p4f2_gn_201101010300-201201010000.nc'

CanESM5: rsdscs, 199101010130-200012312230.nc, 3hr

In [41]:
var_lis_group = ['rsds','rsdscs','mrro']

In [42]:
example_url = 'http://crd-esgf-drc.ec.gc.ca/thredds/fileServer/esgA_dataroot/AR6/CMIP6/CMIP/CCCma/CanESM5/historical/r1i1p2f1/3hr/rsdscs/gn/v20190429/rsdscs_3hr_CanESM5_historical_r1i1p2f1_gn_199101010130-200012312230.nc'

CanESM5:rsds, 199101010130-200012312230.nc, 3hr

In [2]:
var_lis_group = ['rsds', 'rsdscs', 'mrro']

In [3]:
example_url = 'http://crd-esgf-drc.ec.gc.ca/thredds/fileServer/esgA_dataroot/AR6/CMIP6/CMIP/CCCma/CanESM5/historical/r1i1p2f1/3hr/rsds/gn/v20190429/rsds_3hr_CanESM5_historical_r1i1p2f1_gn_199101010130-200012312230.nc'

In [4]:
time_stamp = example_url.split('_')[-1].split('.')[0].split('-')
start_year, end_year = time_stamp[0], time_stamp[1]
start_year, end_year

('199101010130', '200012312230')

不跨年版本

In [12]:
time_period_lis = [str(i) for i in range(1979, 2016)]
time_endpoint = []
for i in time_period_lis:
    tp = i + '01010130' + '-' + i + '12312230.nc'
    time_endpoint.append(tp)
time_endpoint[0], time_endpoint[-1]

('197901010130-197912312230.nc', '201501010130-201512312230.nc')

跨年版本

In [44]:
time_period_lis = {str(i): str(j) for i,j in zip(range(1979,2015),range(1980,2016))}
#201101010300-201201010000.nc
time_endpoint = []

for k,v in time_period_lis.items():
    end_p= k + '01010130' + '-' + v + '12312230.nc'
    time_endpoint.append(end_p)
time_endpoint[0], time_endpoint[-1]


('197901010130-198012312230.nc', '201401010130-201512312230.nc')

跨十年版本

In [8]:
# 196101010300-197101010000.nc
time_period_lis = {str(1971+i*10):str(1971+(i+1)*10) for i in range(4)}
time_endpoint=[]

for k, v in time_period_lis.items():
    tp = k+'01010130'+'-'+v+'12312230.nc'
    time_endpoint.append(tp)
time_endpoint.append('201101010130-201412312230.nc')
time_endpoint[0], time_endpoint[-2], time_endpoint[-1]

# CanESM5
time_endpoint = ['197101010130-198012312230.nc','198101010130-199012312230.nc',
 '199101010130-200012312230.nc','200101010130-201012312230.nc','201101010130-201412312230.nc']

对于sspxxx情形

In [56]:
import logging
def set_logging(log_file_path='logs/transfer_txt.log'):
    os.makedirs(os.path.dirname(log_file_path), exist_ok=True)
    # 创建日志记录器
    logger = logging.getLogger('transfer_txt_logger')
    logger.setLevel(logging.INFO)
    
    # 避免重复添加处理器
    if not logger.handlers:
        # 创建文件处理器
        file_handler = logging.FileHandler(log_file_path, encoding='utf-8')
        file_handler.setLevel(logging.INFO)
        
        # 创建控制台处理器
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        
        # 创建格式器并添加到处理器
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)
        console_handler.setFormatter(formatter)
        
        # 添加处理器到记录器
        logger.addHandler(file_handler)
        logger.addHandler(console_handler)
    
    return logger
    
logger = set_logging()

In [5]:
import tqdm
for var in tqdm.tqdm(var_lis_group):
    variant = var[0]
    scenario = var[1]
    
    url_lis = []
    for i in range(len(time_endpoint)):
        url = example_url.replace('200501010130-200512312230.nc',time_endpoint[i])
        url = url.replace('huss', str(variant))
        url = url.replace('ssp126', str(scenario))

        url_lis.append(url)
    #print(url_lis[0], url_lis[-1])

    dir = 'G:\\MIROC-ES2H\\{}\\{}'.format(variant, scenario)
    import os
    txt_name = os.path.join(dir, '{}_{}.txt'.format(variant, scenario))
    if os.path.exists(txt_name):
        print('文件已经存在',txt_name)
    else:
        with open(txt_name, 'w') as f:
            for i in url_lis:
                f.write(i+'\n')
        print('保存成功',txt_name)


  0%|          | 0/3 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'G:\\MIROC-ES2H\\r\\s\\r_s.txt'

对于historical

In [9]:
import tqdm
for var in tqdm.tqdm(var_lis_group):
    variant = var
    scenario = 'historical'
    url_lis = []
    for i in range(len(time_endpoint)):
        url = example_url.replace('199101010130-200012312230.nc',time_endpoint[i])
        url = url.replace('huss', str(variant))

        url_lis.append(url)
    #print(url_lis[0], url_lis[-1])

    dir = 'G:\\CanESM5-historical\\{}\\{}'.format(variant, scenario)
    import os
    txt_name = os.path.join(dir, '{}_{}.txt'.format(variant, scenario))
    os.makedirs(dir, exist_ok=True)
    with open(txt_name, 'w') as f:
        for i in url_lis:
            f.write(i+'\n')
    print('保存成功',txt_name)
    '''if not os.path.exists(txt_name):
        with open(txt_name, 'w') as f:
            for i in url_lis:
                f.write(i+'\n')
        print('保存成功',txt_name)
    else:
        print('文件已经存在',txt_name)'''


100%|██████████| 3/3 [00:00<00:00, 61.22it/s]

保存成功 G:\CanESM5-historical\rsds\historical\rsds_historical.txt
保存成功 G:\CanESM5-historical\rsdscs\historical\rsdscs_historical.txt
保存成功 G:\CanESM5-historical\mrro\historical\mrro_historical.txt


In [13]:
def get_txt_path(root_dir):
    '''
    Return:save_dir_load_txt| dict
        k: 保存目录，确保存在；
        v: txt文件路径'''

    save_dir_load_txt={}

    for dirpath, dirnames, filenames in os.walk(root_dir):
        if len(filenames) > 0:
            for fl in filenames:
                if fl.endswith('.txt') and fl.split('_')[1] == 'historical.txt':
                    save_dir_load_txt[f'{dirpath}']=(os.path.join(dirpath,fl))
    return save_dir_load_txt
print(get_txt_path('G:\\MIROC-ES2H'))

{'G:\\MIROC-ES2H\\huss\\historical': 'G:\\MIROC-ES2H\\huss\\historical\\huss_historical.txt', 'G:\\MIROC-ES2H\\tas\\historical': 'G:\\MIROC-ES2H\\tas\\historical\\tas_historical.txt', 'G:\\MIROC-ES2H\\uas\\historical': 'G:\\MIROC-ES2H\\uas\\historical\\uas_historical.txt', 'G:\\MIROC-ES2H\\vas\\historical': 'G:\\MIROC-ES2H\\vas\\historical\\vas_historical.txt'}


In [ ]:
import os
def get_txt_path(root_dir):
    '''
    Return:save_dir_load_txt| dict
        k: 保存目录，确保存在；
        v: txt文件路径'''

    save_dir_load_txt={}

    for dirpath, dirnames, filenames in os.walk(root_dir):
        if len(filenames) > 0:
            for fl in filenames:
                if fl.endswith('.txt') and fl.split('_')[0] not in ['huss','tas','uas','vas']:
                    save_dir_load_txt[f'{dirpath}']=(os.path.join(dirpath,fl))
    return save_dir_load_txt
get_txt_path(root_dir = 'G:\\MIROC-ES2H')

mrro_ssp245.txt
mrro_ssp585.txt
rsds_ssp126.txt
rsds_ssp245.txt
rsds_ssp585.txt
rsdscs_ssp126.txt
rsdscs_ssp245.txt
rsdscs_ssp585.txt


{'G:\\MIROC-ES2H\\mrro\\ssp245': 'G:\\MIROC-ES2H\\mrro\\ssp245\\mrro_ssp245.txt',
 'G:\\MIROC-ES2H\\mrro\\ssp585': 'G:\\MIROC-ES2H\\mrro\\ssp585\\mrro_ssp585.txt',
 'G:\\MIROC-ES2H\\rsds\\ssp126': 'G:\\MIROC-ES2H\\rsds\\ssp126\\rsds_ssp126.txt',
 'G:\\MIROC-ES2H\\rsds\\ssp245': 'G:\\MIROC-ES2H\\rsds\\ssp245\\rsds_ssp245.txt',
 'G:\\MIROC-ES2H\\rsds\\ssp585': 'G:\\MIROC-ES2H\\rsds\\ssp585\\rsds_ssp585.txt',
 'G:\\MIROC-ES2H\\rsdscs\\ssp126': 'G:\\MIROC-ES2H\\rsdscs\\ssp126\\rsdscs_ssp126.txt',
 'G:\\MIROC-ES2H\\rsdscs\\ssp245': 'G:\\MIROC-ES2H\\rsdscs\\ssp245\\rsdscs_ssp245.txt',
 'G:\\MIROC-ES2H\\rsdscs\\ssp585': 'G:\\MIROC-ES2H\\rsdscs\\ssp585\\rsdscs_ssp585.txt'}